# GOES-R Solar Irradiance
The GOES-R series have been monitoring atmospheric conditions and space weather since 1975. They are operated by the National Oceanic and Atmospheric Administration (NOAA) with the support of the National Aeronautic and Space Administration (NASA). 
https://www.goes-r.gov/

## About
[GOES-R Extreme Ultraviolet and X-ray Irradiance Sensors (EXIS)](https://www.ncei.noaa.gov/products/goes-r-extreme-ultraviolet-xray-irradiance)
>The Geostationary Operational Environmental Satellite (GOES)-R Extreme Ultraviolet and X-Ray Irradiance Sensors (EXIS) suite consists of two main instruments, the Extreme Ultraviolet Sensor (EUVS) and the X-Ray Sensor (XRS). EUVS products include high-cadence and time-averaged measurements of irradiance for seven extreme ultraviolet (EUV) solar lines, the magnesium II core-to-wing ratio (Mg II index), and modeled EUV spectra. 


# Extreme Ultraviolet Sensors (EUVS) Data

## Lyman-Alpha (EUV Proxy)
The **Lyman-Alpha (121.6nm)** is the brightest solar wavelength in the ultraviolet. It is emitted by the hydrogen in the Sun's atmosphere. It has been found to have corralation to the square root of the F10.7 index. 

### How Lyman-Alpha corralates to the F10.7 index

>"The second model (used when the Mg II index is not available) is based on the 10.7‐cm solar radio flux (F10.7) measurements made near Ottawa and Penticton (Canada) (Tapping, 2013). The Lyman α composite goes back to 1947 with the use of the F10.7 proxy... The F10.7 proxy is based on the square root of the F10.7, because the square root of the F10.7 has a better correlation with the Lyman α measurements."
>- Machol, J., Snow, M., Woodraska, D., Woods, T., Viereck, R., & Coddington, O. (2019). An improved lyman‐alphacomposite. Earth and Space Science, 6, 2263–2272. https://doi.org/10.1029/2019EA000648

## GOES-15
- **Source:** [NOAA](https://www.ncei.noaa.gov/products/goes-1-15/space-weather-instruments)
- **File:** G15_EUVE_daily_2010_2016_v4.txt
- **Product:**  1-min & Daily Averages (version 4)
     - EUVS channel E (121.6nm) 1-min and 1-day averaged counts, irradiances and flags
- **Date:** 2010-2016

**Note:**
we will need to  multiple the degration corrected average irradiance (irrad_ly) by AU_corr to correct the irradiance given that the distance between the Earth and Sun varies throughout the year. 

In [6]:
import pandas as pd


path = "datasets/G15_EUVE_daily_2010_2016_v4.txt"

df = pd.read_csv(
    path,
    comment=";",
    sep=r"\s+",
    header=None,
    names=["date", "Julday", "counts", "flags", "num", "irrad", "irrad_ly", "au_corr"],
    na_values=["-999.0", "-999"],
    dtype=str
)


# Change irrad_ly and au_corr to numeric
df[["irrad_ly", "au_corr"]] = df[["irrad_ly", "au_corr"]].apply(pd.to_numeric, errors="coerce")

# Create datetime column
df["date"] = pd.to_datetime(df["date"], errors="coerce")

#1 AU correction
df = pd.eval("irrad_corr = df.irrad_ly * df.au_corr", target=df)

# Drop bad/missing data
df = df.dropna().reset_index(drop=True)

# Drop uneccessary column
df = df.drop(["Julday", "counts", "flags", "num", "irrad",], axis=1)

#save as csv
df.to_csv("datasets/GOES15_average_irradiance.csv", index=False)

print(df)

           date  irrad_ly   au_corr  irrad_corr
0    2010-04-07  0.006309  1.000411    0.006312
1    2010-04-08  0.006492  1.000987    0.006498
2    2010-04-09  0.006475  1.001563    0.006485
3    2010-04-10  0.006446  1.002138    0.006460
4    2010-04-11  0.006424  1.002714    0.006441
...         ...       ...       ...         ...
2195 2016-06-02  0.006737  1.028364    0.006928
2196 2016-06-03  0.006713  1.028679    0.006906
2197 2016-06-04  0.006695  1.028986    0.006889
2198 2016-06-05  0.006666  1.029284    0.006861
2199 2016-06-06  0.006651  1.029572    0.006848

[2200 rows x 4 columns]


/var/folders/ms/0f33lp7x7wjc5dbt948jwyvr0000gn/T/ipykernel_32307/1421109324.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(df["date"], errors="coerce")


# GOES-16
- **Source:** [NOAA](https://www.ncei.noaa.gov/products/goes-r-extreme-ultraviolet-xray-irradiance)
- **File:** sci_euvs-l2-avg1d_g16_s20170207_e20250406_v1-0-6.nc
- **Product:**  EUVS Daily Average
     - Daily averages of spectral line irradiances, the Mg II index, and proxy spectra.
- **Date:** 2017-2025

In [7]:
import netCDF4 as nc
import xarray as xr

path = "datasets/sci_euvs-l2-avg1d_g16_s20170207_e20250406_v1-0-6.nc"

# see variables we are interested in
file = nc.Dataset(path)
print(file.variables['irr_1216'])
print("\n")
print(file.variables['MgII_standard'])
print("\n")
print(file.variables['au_factor'])

<class 'netCDF4.Variable'>
float32 irr_1216(time)
    _FillValue: -9999.0
    long_name: Daily average irradiance for 121.6-nm line.
    comments: Average excludes hours of major geocoronal impact.
    units: W/m2
    valid_min: 0.004
    valid_max: 0.2
unlimited dimensions: time
current shape = (2981,)
filling on


<class 'netCDF4.Variable'>
float32 MgII_standard(time)
    _FillValue: -9999.0
    long_name: Daily average of EXIS Mg II index scaled to  a standard Mg II measurement resolution.
    valid_min: 0.0
    valid_max: 1.0
unlimited dimensions: time
current shape = (2981,)
filling on


<class 'netCDF4.Variable'>
float32 au_factor(time)
    _FillValue: 0.0
    long_name: 1-AU factor.
    comments: Multiplicative factor to convert fluxes to values at 1 AU.
unlimited dimensions: time
current shape = (2981,)
filling on


In [8]:
# convert to dataframe
ds = xr.open_dataset(path)
df = ds.to_dataframe()

# flatten dataset (one sample per a day)
df_time = df.groupby('time').mean(numeric_only=True)

# assign features & rename
features = df_time[['irr_1216', 'MgII_standard', 'au_factor']]
features = features.rename(columns={
    'irr_1216': 'Lyman_alpha',
    'MgII_standard': 'MgII'
})

features.head(5)

,Lyman_alpha,MgII,au_factor
time,,,
2017-02-07,0.006339,0.262891,0.972852
2017-02-08,0.006352,0.263016,0.973178
2017-02-09,0.006431,0.263391,0.973516
2017-02-10,0.006431,0.263411,0.973863
2017-02-11,0.006484,0.263713,0.974222


In [9]:
import numpy as np

# Change columns to numeric
features[['Lyman_alpha', 'MgII', 'au_factor']] = features[['Lyman_alpha', 'MgII', 'au_factor']].apply(pd.to_numeric, errors="coerce")

# Assign datetime column
features = features.reset_index() # make time a col not index
features["time"] = pd.to_datetime(features["time"], errors="coerce")

#1 AU correction
features = pd.eval("Lyman_alpha_corr = features.Lyman_alpha * features.au_factor", target=features)
features = pd.eval("MgII_corr = features.MgII * features.au_factor", target=features)

# Drop bad/missing data
features[['Lyman_alpha', 'MgII', 'au_factor']] = features[['Lyman_alpha', 'MgII', 'au_factor']].replace([-9999.0, 0], np.nan)
features = features.dropna().reset_index(drop=True)

#save as csv
df.to_csv("datasets/GOES16_average_irradiance.csv", index=False)

features.head(5)


,time,Lyman_alpha,MgII,au_factor,Lyman_alpha_corr,MgII_corr
0,2017-02-07,0.006339,0.262891,0.972852,0.006166,0.255754
1,2017-02-08,0.006352,0.263016,0.973178,0.006181,0.255961
2,2017-02-09,0.006431,0.263391,0.973516,0.006260,0.256415
3,2017-02-10,0.006431,0.263411,0.973863,0.006263,0.256527
4,2017-02-11,0.006484,0.263713,0.974222,0.006317,0.256915


# X-Ray Sensors Data

## Solar Flares

## GOES-15
- **Source:** [NOAA](https://www.ncei.noaa.gov/products/goes-1-15/space-weather-instruments)
- **File:** sci_xrsf-l2-flsum_g15_s20100407_e20200304_v2-3-0.nc
- **Product:**  Flare Summary
     - List of solar flares with times, flare classes and integrated fluxes
- **Date:** 2010-2020
